# Lokale depositiebijdrage van N- emissiebronnen mbv Aerius / OPS

Om kwetsbare natuur te beschermen, zal de N-depositie in gevoelige N2000-gebieden moeten verminderen. Voor het treffen van maatregelen is het van belang om de emissiebronnen die voor (een grote) depositie zorgen, zo goed mogelijk in beeld te krijgen. Deze informatie maakt het mogelijk om een helder beeld te krijgen van het nut van verschillende maatregelpaketten

In Aerius monitor valt te achterhalen welke sectoren de grootste bijdrage leveren aan de depositie. Er valt echter niet te achterhalen wat de ruimtelijke spreiding van deze emissiebronnen is, of hoeveel invloed de individuele bronnen hebben. 

Dit script helpt bij het berekenen van de lokale effecten van stikstofemissies. 

# Voorwerk: Emissiebronnen (p,l,v) &rarr; Emissieraster (500m x 500m)

De eerste stap is aggregeren van de emissiebronnen naar een 500m2 (vector) raster. Dit heeft als voordeel dat de rekentijd in Aerius aanzienlijk korter wordt en dat de brondata geanonimiseerd kan worden weergegeven. 

Vanuit de brondata genereren we een csv/gpkg/shape oid met in ieder geval de volgende velden:

| Variable         | Voorbeeld | Type      | Eenheid | Aerius | Opmerking                                              |
| ---------------- | --------- | --------- | ------- | ------ | -------------------------------------------------------|
| x                | 90250     | Integer   | m       | -      | X-coordinaat middelpunt rastercel in EPSG:28992        |
| y                | 427250    | Integer   | m       | -      | Y-coordinaat middelpunt rastercel in EPSG:28992        |
| q                | 1.852E-01 | Decimalen | g/s     | -      |                                                        |
| hc               | 0.4       | Decimalen | MW      | -      | Warmteinhoud bron                                      |
| h                | 12        | Decimalen | m       | -      | Gemiddelde uitstoothoogte bron                         |
| r                | 500       | Integer   | m       | -      | Diameter bron                                          |
| s                | 6.0       | Decimalen | m       | -      | Spreiding. Als spread > hoogte, dan spread = hoogte    |
| dv               | 2         | Integer   | -       | -      | Temporele variatie bron                                |
| cat              | 4320      | Integer   | -       | -      | Sector ID. Bij onbekend wordt sector industrie ingevuld|
| area             | 1         | Integer   | -       | -      | Gebiedsnummer. Nederland = 1                           |
| ps               | 0         | Integer   | -       | nee    | Particle size distribution                             |
| component        | NOx       | Text      | -       | nee    | NOx of NH3                                             |
| bronomschrijving | Test      | Text      | -       | nee    | Omschrijving.                                          |
| ai_code          | 07654495  | Integer   | -       | nee    | Uniek ID van emissieregistratie 500m rastercel         |

Per sector zijn standaardwaarden te vinden via de Aerius [website](https://www.aerius.nl/nl/factsheets/bronkenmerken-sectoren-gcngdn/01-07-2015). De pdf is via deze [link](https://www.aerius.nl/files/media/factsheets/bronkenmerken_sectoren_aerius_calculator_-_7_september_2015.pdf) te downloaden. Het proces om de rasters te genereren vanuit de emissiebronnen houden we voor nu buiten de scope.

# Configuratie

Het script gaat uit van de volgende folderstructuur:

```
root
├── emissiegrids             --> Zelf aanmaken (hier staan de 500m2 emissiegrids)
│   ├── wonen.gpkg             
│   ├── glastuinbouw.gpkg
│   └── etc.
├── rekendata                --> Wordt aangemaakt
│   ├── glastuinbouw
│   │    ├── brn
│   │    ├── emissie_gml
│   │    └── resultaat_gml
│   └── etc.
├── bronnen-lokaliseren.ipynb 
├── utils.py                 --> Helper functies
└── aerius-env.yml           --> yml bestand met de benodigde packages
```

In het conda-env.yml bestand staan alle benodigde packages. Installatie gaat gemakkelijk via conda en het command:
```sh
conda env create -f aerius-env.yml
```
De packages kunnen ook handmatig worden geinstalleerd. Met minimaal python3, fiona, pandas, geopandas, shapely, requests zou alles moeten werken.

# API-key aanvragen & overige config

API key aanvragen via Aerius Connect.

In [ ]:
from utils import generate_api_key

email = input('Voer je mailadres in: ')
resp = generate_api_key(email)

print('Json response: {}'.format(resp.json()))

De API Key is naar het opgegeven mailadres verzonden. 
De API key, de sector, rekenstof en rekenjaar vullen we in.

In [ ]:
sector = "Glastuinbouw"        # Glastuinbouw
rekenstof = "NOX"              # NOX of NH3 -> hoofdlettergevoelig
year = 2018                    # Rekenjaar
apiKey = ' ' # Staat in de mail na aanvraag

Als de folderstructuur niet bestaat, dan genereren we hem.

In [ ]:
from pathlib import Path

# Bouw de folder structuur als die nog niet bestaat
for folder in ['emissie_gml', 'brn', 'depositie_gml', 'shp', 'csv']:
    Path(r'.\rekendata\{}\{}'.format(sector, folder)).mkdir(parents=True, exist_ok=True)

# Stap 1: Emissieraster &rarr; OPS invoerbestand(en) &rarr; Emissie-GML

Uit het emissieraster genereren we Emissie-GML's waarmee Aerius kan rekeken. Omdat we de lokale invloed willen bepalen, doen we dit per emissiecel.

1. Eerst lezen we het emissieraster in. (*Let op, het is een vector raster*) De package 'fiona' is hiervoor geschikt. Mbv fiona kunnen we itereren over de features (emissierastercellen) in het ruimtelijke bestand (het emissieraster). 
2. We check eerst of de rastercel emissies bevat
3. Per rastercel genereren we een OPS bronbestand (.brn).
4. Dit .brn bestand sturen we naar Aerius Connect om het te laten converteren in een rekenbare GML.
5. Als laatst passen we het rekenjaar van de GML aan, en schrijven we het bestand weg.

_ToDo: stap 3+4+5 kunnen ook rechstreeks._

##### LET OP: dit duurt lang..

In [ ]:
import os
import fiona 
from utils import write_brn, convert_brn_to_gml 

# De locatie van de geopackage met het vector raster
filePath = r".\emissiegrids\{}.gpkg".format(sector)
tempPath = r".\rekendata\{}".format(sector)

# 1. Itereer over geopackage features. (de rastercellen)
for i, record in enumerate(fiona.open(filePath, 'r'), 1):
    
    # 2. Check of de cel uberhaupt emissies bevat
    # print(record['properties']['q'], record['properties']['id'], type(record['properties']['q']))
    if (record['properties']['q'] == 0) or (record['properties']['q'] is None):
        print('AI CODE: {} .. Geen emmissies'.format(record['properties']['ai_code']))
        i =- 1
        continue
        
    # 2. Per rastercel genereren we een BRN bestand. 
    # Fiona leest features als ordered dict. Converteren naar dict.
    rd = dict(record['properties'].items())   
    brn = write_brn(rd, tempPath)
    
    # 3. Als het bestand nog niet is geconverteerd, 
    # dan sturen we het naar connect.
    targetGMLPath = r'.\rekendata\{}\emissie_gml\{}.gml'.format(sector, rd['ai_code'])
    if os.path.exists(targetGMLPath) == True: 
        print('Bestand bestaat')
        continue
    else:
        print('Stuur cel naar connect voor convertie')
        resp = convert_brn_to_gml(brn, rekenstof).json()
        
    # 4. GML wegschrijven met aangepast rekenjaar.    
    with open(targetGMLPath, "w") as gml:
        imaerGml = resp['dataObject']['data'] 
        imaerGml = imaerGml.replace("year>0</i", "year>{}</i".format(year))
        gml.write(imaerGml)
        print(i, targetGMLPath)
        

## Stap 2: Depositieberekening per cel via Aerius Connect

1. We itereren door het GML bestand
2. Er mogen max 6 berekeningen tegelijk naar Connect gaan. We checken eerst of de wachtrij vol zit, zo ja, dan wachten we 6 sec en checken we het opnieuw.
3. We gaan door de lijst van jobs. Als een berekening afgerond is, dan downloaden we het zip bestand en pakken het uit (met de naam van het rastercel). 

##### LET OP: dit duurt _erg_ lang..

In [ ]:
import glob, os, time, random
from utils import get_job_status, download_and_unzip, calculate_job

startTime = time.strptime('2020-01-07 12:00', '%Y-%m-%d %H:%M') # Omdat de API niet voorziet in filtering van de jobs, is het handig om dat zelf te doen.
tempPath = r".\rekendata\{}".format(sector)

# Lijst van al berekende GMLs
globString1 = r'.\rekendata\{}\depositie_gml\*.gml'.format(sector)
completed = [os.path.basename(x) for x in glob.glob(globString1)]

# Lijst van alle te berekenen GMLs
globString2 = r'.\rekendata\{}\emissie_gml\*.gml'.format(sector)
todo = [os.path.basename(x) for x in glob.glob(globString2)]
todoL = len(todo)

# Haal de afgeronde items van de todo-lijst af
todo = [item for item in todo if item not in completed]
print('{} To do.. '.format(len(todo)))

# Per GML emissiebestand
for i, gml in enumerate(todo):
    running = []
    jobs = get_job_status(apiKey).json()['entries'] 
  
    # Wacht tot er plek is voor een nieuwe GML (max 6 tegelijk)
    while str(jobs).count('RUNNING') > 5:
        jobs = get_job_status(apiKey).json()['entries']
        print('--')
        time.sleep(10)      

    # Download completed berekeningen, log de gmls die nu
    # staan te rekenen (zodat ze niet nogmaals worden verstuurd)
    for job in jobs:
        jobTime = time.strptime(job['startDateTime'][:-12], '%Y-%m-%dT%H:%M')
        if job['state'] == 'COMPLETED' and jobTime > startTime and \
           job['name'] not in completed:
            try:
                print("Download: {}".format(job['name']))
                download_and_unzip(job['name'], job['resultUrl'], tempPath)
                completed.append(job['name'])
            except:
                print("----> failed")                
        elif job['state'] == 'RUNNING' or job['state'] == 'QUEUED':
            running.append(job['name'])  
    
    # Als de GML niet momenteel wordt berekend, of dat al is,
    # dan lezen we de GML en sturen we hem naar Connect
    if gml not in completed and gml not in running:
        
        # Lees de GML
        gmlPath = r'.\rekendata\{}\emissie_gml\{}'.format(sector,gml)     
         
        with open(gmlPath, 'r') as myfile:
            gmlFile = myfile.read()    
            print(calculate_job(apiKey, gmlFile, gml, year))
        
        print('Progress: completed: {} '\
              'running: {} total: {}'.format(len(completed), len(running), todoL))
        time.sleep(2)
    else:
        print('{}/{} ::{} completed or running'.format(i, todoL, gml))
        

Dit stukje code vraagt de huidige status van de berekeningen op:

In [ ]:
import time
from utils import get_job_status

# Omdat de API niet voorziet in filtering van de jobs, is het handig om dat zelf te doen.
# Pas deze waarde aan 
startTime = time.strptime('2020-01-07 12:00', '%Y-%m-%d %H:%M') 

for job in get_job_status(apiKey).json()['entries'] :
    jobTime = time.strptime(job['startDateTime'][:-12], '%Y-%m-%dT%H:%M')
    
    if jobTime > startTime:
        print(job['name'], job['state'], job['key'])
    

## Stap 3: Verwerken depositieberekening

Volgend uit stap 2 hebben we nu een folder met enkele duizenden depositieberekeningen (in de vorm van GMLs). Elke GML bevat depositieberekening van de emissies van een bepaalde cel. De naam van het GML-bestand correspondeert met de code van de Emissie-cel.

De data schrijven we weg als CSV met het volgende format:

Emissievlak (1), Receptor (2), Depositieresultaat (3), Geometrie (4)

1. De code (ai) van het emissievlak vanwaar de deposities zijn berekend.
2. De code (rcp) van de receptor waar de depositie neerslaat.
3. De berekende depositie (dep) op dit receptorpunt.
4. De geometrie van het hexagoon in WKT format.


[TODO: wegschrijven naar een (lokale) database of ander efficienter format]*

### _Uitlezen depositieberekening GML's_

In [ ]:
fname = 'Resultaat_{}.csv'.format(sector)

In [ ]:
import utils
import csv
import glob, os

from importlib import reload
reload(utils)

# Lijst van GMLs met rekenresultaten
gml_files = glob.glob(r".\rekendata\{}\depositie_gml\*.gml".format(sector))
result = []


# Loop door de resultaten heen
for i, gml in enumerate(gml_files):
        
    base = os.path.basename(gml)
    base_no_ext = os.path.splitext(base)[0]
    print('{} GML no.: {}'.format(i, base_no_ext))
    
    for record in utils.yield_receptor_data(gml):
        result.append({'ai' : base_no_ext, 'rcp': record[0], 'dep': record[1], 'wkt': record[2]})     
        
try:
    with open(fname, 'w') as csv:
        csv.write('AI; RCP; DEP; RCP_WKT\n')
        for i, val in enumerate(result):
            csv.write('{ai}; {rcp}; {dep}; {wkt}\n'.format(**val)) 
    print('CSV saved to: {}\{}'.format(os.getcwd(), fname)) 
except:
    print('Cant save to csv')

### _Joinen grid-geometrieen (AI CODES) & n2000 gebieden._

Vervolgens koppelen we de receptoren aan de Natura2000-gebieden waar ze binnen vallen.

In [ ]:
import pandas as pd
import geopandas as gpd
import shapely.wkt

# Inlezen CSVs en shapefile
depositie_resultaten = pd.read_csv(fname, delimiter="; ", engine='python').set_index('AI')
emissie_raster = pd.read_csv("emissiegrids/Emissieraster_leeg.csv", delimiter=";", engine='python').set_index('AI_CODE')
n2000 = gpd.read_file("/overige_datasets/n2k_b/n2k_buffer.shp")

# Join 
dep_join_ai = depositie_resultaten.join(emissie_raster)

# Lees geometrie uit WKT string, maak geopandas dataframe in RD
geometry = dep_join_ai['RCP_WKT'].map(shapely.wkt.loads)
dep_gdf = gpd.GeoDataFrame(dep_join_ai, crs={'init':'epsg:28992'}, geometry=geometry)

# Pak middelpunt van receptorpunt, spatial join met n2000
dep_gdf['geometry'] = dep_gdf['geometry'].centroid
joined_gdf = gpd.sjoin(dep_gdf, n2000, how='left', op='intersects', lsuffix='left_', rsuffix='right_')

# Door de buffer in 't n2000 bestand ontstaan er dubbele joins. Verwijder dubbele matches.
joined_gdf_no_duplicates = joined_gdf.drop_duplicates(subset=['AI_WKT', 'RCP'], keep='first')

Als output zijn we geïnteresseerd in twee type bestanden.

De éne bevat de geometrie van de receptoren, de andere de geometrie van de emissievlakken.

In [ ]:
# Bestand 1: AI_code, RCP_id, NOx_dep, n2k_name, geom (van ai_code)
v1_RCP_GEOM = joined_gdf_no_duplicates.rename(index=str, columns={"RCP": "RCP_id", "DEP": "NOx_dep","natura2001": "N2k_name"})
v1_RCP_GEOM = v1_RCP_GEOM.drop(['index_right_', 'AI_WKT', 'geometry'], axis=1)

v1_RCP_GEOM.to_csv('rcp_geometrie.csv')

# Bestand 2: AI_code, RCP_id, NOx_dep, n2k_name, geom (van receptor)
v2_AI_GEOM = joined_gdf_no_duplicates.rename(index=str, columns={"RCP": "RCP_id", "DEP": "NOx_dep","natura2001": "N2k_name"})
v2_AI_GEOM = v2_AI_GEOM.drop(['index_right_', 'RCP_WKT', 'geometry'], axis=1)

v2_AI_GEOM.to_csv('ai_geometrie.csv')
